In [1]:
import numpy as np

import torch
import torchvision
import  torchvision.transforms as transforms
from mean_teacher import data

from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler

In [44]:
a = torch.Tensor([1])

In [51]:
{i:(1 if i%2 == 0 else 0) for i in range(10)}

{0: 1, 1: 0, 2: 1, 3: 0, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 0}

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
train_dir = '/scratch/ehd255/ssl_data_96/supervised/train/'
val_dir = '/scratch/ehd255/ssl_data_96/supervised/val/'
unsup_dir = '/scratch/ehd255/ssl_data_96/unsupervised/'

In [33]:
dataset[0][0][0].size()

torch.Size([3, 96, 96])

In [34]:
dataset[0][0][1].size()

torch.Size([3, 96, 96])

In [43]:
96 * 4

384

In [21]:
dataset = torchvision.datasets.ImageFolder(train_dir, ssl()['train_transformation'])
labeled_idx = list(range(len(dataset)))

_dataset = torchvision.datasets.ImageFolder(unsup_dir, ssl()['train_transformation'])

for i in _dataset.classes:
    _dataset.class_to_idx[i] = -1
    
concat_dataset = torch.utils.data.ConcatDataset([dataset, _dataset])
unlabeled_idxs = list(range(len(dataset), len(dataset) + len(_dataset)))

dataset = concat_dataset

In [89]:
batch_sampler = data.TwoStreamBatchSampler(unlabeled_idxs, 
                                           labeled_idxs, 
                                           args.batch_size, 
                                           args.labeled_batch_size)

train_loader = torch.utils.data.DataLoader(dataset, 
                                           batch_sampler=batch_sampler, 
                                           num_workers=args.workers, 
                                           pin_memory=True)

train_loader_len = len(train_loader)

eval_loader = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(evaldir, eval_transformation),
                                        batch_size=eval_batch_size,
                                        shuffle=False,
                                        num_workers=2 * args.workers,  
                                        pin_memory=True,
                                        drop_last=False)

In [26]:
import os

In [ ]:
runner_file = ""

In [ ]:
os.path.basename(runner_file).split(".")[0]

In [ ]:
def create_data_loaders_ssl(train_transformation, eval_transformation, datadir, args):
    
    traindir = os.path.join(datadir, args.train_subdir)
    evaldir = os.path.join(datadir, args.eval_subdir)
    
    dataset = torchvision.datasets.ImageFolder(train_dir, train_transformation)
    labeled_idx = list(range(len(dataset)))

    _dataset = torchvision.datasets.ImageFolder(unsup_dir, train_transformation)

    for i in _dataset.classes:
        _dataset.class_to_idx[i] = -1

    concat_dataset = torch.utils.data.ConcatDataset([dataset, _dataset])
    unlabeled_idxs = list(range(len(dataset), len(dataset) + len(_dataset)))

    dataset = concat_dataset
    
    batch_sampler = data.TwoStreamBatchSampler(unlabeled_idxs, 
                                           labeled_idxs, 
                                           args.batch_size, 
                                           args.labeled_batch_size)

    train_loader = torch.utils.data.DataLoader(dataset, 
                                               batch_sampler=batch_sampler, 
                                               num_workers=args.workers, 
                                               pin_memory=True)

    train_loader_len = len(train_loader)

    eval_loader = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(evaldir, eval_transformation),
                                            batch_size=eval_batch_size,
                                            shuffle=False,
                                            num_workers=2 * args.workers,  
                                            pin_memory=True,
                                            drop_last=False)
    
    return train_loader, eval_loader, train_loader_len